In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
imdb_reasons = pd.read_pickle('../data/imdb_reasons.pkl')
imdb_reasons.info()

In [ ]:
imdb_reasons['mpaa_cert_#'] = imdb_reasons['mpaa_cert_#'].str.strip()

In [ ]:
# Going to have to deal with these separately
imdb_dup_certs = imdb_reasons.loc[(imdb_reasons.duplicated(subset = ['mpaa_cert_#'], keep = False))
                                  &
                                  (imdb_reasons['mpaa_cert_#'] != 'None')].sort_values('mpaa_cert_#').reset_index(drop=True)
len(imdb_dup_certs)

In [ ]:
imdb_reasons = imdb_reasons.loc[~(imdb_reasons['mpaa_cert_#'].isin(imdb_dup_certs['mpaa_cert_#']))].reset_index(drop=True)

In [ ]:
mpaa_details = pd.read_pickle('../data/mpaa_details.pkl')
mpaa_details.info()

In [ ]:
mpaa_details['mpaa_cert_#'] = mpaa_details['mpaa_cert_#'].str.strip()

In [ ]:
# Going to have to deal with these separately
mpaa_dup_certs = mpaa_details.loc[mpaa_details.duplicated(subset = ['mpaa_cert_#'], keep = False)].sort_values('mpaa_cert_#').reset_index(drop=True)
len(mpaa_dup_certs)

In [ ]:
mpaa_details = mpaa_details.loc[~(mpaa_details['mpaa_cert_#'].isin(mpaa_dup_certs['mpaa_cert_#']))].reset_index(drop=True)

#### Merge the two main datasets

In [ ]:
outer_merge = pd.merge(imdb_reasons, mpaa_details, on = 'mpaa_cert_#', how = 'outer')

In [ ]:
# Amazing Stories was a TV show, not sure why it got rated by MPAA, dropping those rows
outer_merge = outer_merge.loc[~(outer_merge['mpaa_title'].fillna('').str.contains('Amazing Stories'))]

In [ ]:
imdb_no_match = outer_merge.loc[outer_merge['mpaa_title'].isna()]
mpaa_no_match = outer_merge.loc[outer_merge['imdb_ids'].isna()]

In [ ]:
imdb_info = imdb_reasons.loc[imdb_reasons['imdb_ids'].isin(imdb_no_match['imdb_ids'])]
mpaa_info = mpaa_details.loc[mpaa_details['mpaa_cert_#'].isin(mpaa_no_match['mpaa_cert_#'])]

In [ ]:
print(len(imdb_info))
print(len(mpaa_info))

In [ ]:
# These movies originally released prior to the 1992-2022 timeframe except for one that was released in 2023
mpaa_info.loc[mpaa_info['other_info'].str.contains('Re-Rating')].sort_values('rating_year')

In [ ]:
# These are all older movies that were originally released prior to rating reasons being required, will drop them
mpaa_info.loc[mpaa_info['other_info'].str.contains('Re-Issue')]

In [ ]:
# These are also all older movies that were re-rated to use the current rating system
mpaa_info.loc[mpaa_info['other_info'].str.contains('Rating Symbol')]

In [ ]:
outer_merge = outer_merge.loc[~(((outer_merge['other_info'].str.contains('Re-Rating'))
                            |
                             (outer_merge['other_info'].str.contains('Re-Issue'))
                             |
                             (outer_merge['other_info'].str.contains('Rating Symbol')))
                            &
                            (outer_merge['imdb_ids'].isna()))].reset_index(drop=True)

In [ ]:
imdb_info.loc[(imdb_info['rating_reasons'] == 'None')
              & 
              (imdb_info['mpaa_cert_#'] == 'None')
              &
              (~imdb_info['release_note'].isna())]

#After a looking at a selection of the movies, they do not appear to have been rated by the MPAA, so removing them
imdbids_to_remove_a = imdb_info.loc[(imdb_info['rating_reasons'] == 'None')
              & 
              (imdb_info['mpaa_cert_#'] == 'None')
              &
              (~imdb_info['release_note'].isna())]['imdb_ids']

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_a))]

In [ ]:
imdb_info.loc[imdb_info['rating_reasons'] == '']

# none of these 3 were rated by the MPAA despite IMDB having a certificate # for them
imdbids_to_remove_b = imdb_info.loc[imdb_info['rating_reasons'] == '']['imdb_ids']

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_b))]

In [ ]:
imdb_info.loc[(imdb_info['imdb_genres'].str.contains('Short'))
                & 
                (imdb_info['rating_reasons'] == 'None')]

# After a looking at a selection of the movies, they do not appear to have been rated by the MPAA, so removing them
imdbids_to_remove_c = imdb_info.loc[(imdb_info['imdb_genres'].str.contains('Short'))
                                    & 
                                    (imdb_info['rating_reasons'] == 'None')]['imdb_ids']

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_c))]

In [ ]:
imdb_no_match_b = outer_merge.loc[outer_merge['mpaa_title'].isna()]
mpaa_no_match_b = outer_merge.loc[outer_merge['imdb_ids'].isna()]

In [ ]:
imdb_info_b = imdb_reasons.loc[(imdb_reasons['imdb_ids'].isin(imdb_no_match_b['imdb_ids']))]
mpaa_info_b = mpaa_details.loc[mpaa_details['mpaa_cert_#'].isin(mpaa_no_match_b['mpaa_cert_#'])]

In [ ]:
print(len(imdb_info_b))
print(len(mpaa_info_b))

In [ ]:
imdb_info_b.loc[(imdb_info_b['imdb_ratings'].isna())
               & 
              (imdb_info_b['rating_reasons'] == 'None')]

# After a looking at a selection of the movies, they do not appear to have been rated by the MPAA, so removing them
imdbids_to_remove_d = imdb_info_b.loc[(imdb_info_b['imdb_ratings'].isna())
                                      & 
                                      (imdb_info_b['rating_reasons'] == 'None')]['imdb_ids']

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_d))]

In [ ]:
imdb_info_b.loc[(imdb_info_b['release_year'] == '1991')
                &
                (imdb_info_b['rating_reasons'] == 'None')]


# These movies were rated prior to reasons being required
imdbids_to_remove_e = imdb_info_b.loc[(imdb_info_b['release_year'] == '1991')
                                      &
                                      (imdb_info_b['rating_reasons'] == 'None')]['imdb_ids']

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_e))]

In [ ]:
imdb_info_b.loc[(imdb_info_b['mpaa_cert_#'] != 'None')
                &
                (imdb_info_b['rating_reasons'] == 'None')]

# These movies do not align with any MPAA rated movies OR they were rated prior to rating reasons
imdbids_to_remove_f = imdb_info_b.loc[(imdb_info_b['mpaa_cert_#'] != 'None')
                                      &
                                      (imdb_info_b['rating_reasons'] == 'None')]['imdb_ids']

In [ ]:
imdb_no_match_c = outer_merge.loc[outer_merge['mpaa_title'].isna()]
mpaa_no_match_c = outer_merge.loc[outer_merge['imdb_ids'].isna()]

In [ ]:
imdb_info_c = imdb_reasons.loc[(imdb_reasons['imdb_ids'].isin(imdb_no_match_c['imdb_ids']))]
mpaa_info_c = mpaa_details.loc[mpaa_details['mpaa_cert_#'].isin(mpaa_no_match_c['mpaa_cert_#'])]

In [ ]:
print(len(imdb_info_c))
print(len(mpaa_info_c))

In [ ]:
outer_merge = outer_merge.loc[~(outer_merge['imdb_ids'].isin(imdbids_to_remove_f))]

In [ ]:
outer_merge.info()

Now need to deal with the movies that appear multiple times, sometimes with all info, sometimes not

In [ ]:
dup_ids = (
    outer_merge.loc[(outer_merge['imdb_ids'].isin(imdb_info_c['imdb_ids']))
                    &
                    (~outer_merge['mpaa_title'].isna())
                    ]['imdb_ids']
)

In [ ]:
dup_imbdid_indices = (
    outer_merge.loc[(outer_merge['imdb_ids'].isin(dup_ids))
                    &
                    (outer_merge['mpaa_title'].isna())
                    ].index
)

In [ ]:
outer_merge = (
    outer_merge.drop(index=dup_imbdid_indices)
    .reset_index(drop=True)
)

In [ ]:
imdb_no_match_d = outer_merge.loc[outer_merge['mpaa_title'].isna()]
mpaa_no_match_d = outer_merge.loc[outer_merge['imdb_ids'].isna()]

In [ ]:
imdb_info_d = imdb_reasons.loc[(imdb_reasons['imdb_ids'].isin(imdb_no_match_d['imdb_ids']))]
mpaa_info_d = mpaa_details.loc[mpaa_details['mpaa_cert_#'].isin(mpaa_no_match_d['mpaa_cert_#'])]

In [ ]:
print(len(imdb_info_d))
print(len(mpaa_info_d))

Now comes the more complicated efforts to merge the datasets

First, I will save the component of the merged data that worked just using the MPAA certiicate number, removing the remaining portions that didn't merge.

I will then attempt to merge the remaining data (incluiding those elements of each data set that had duplicate MPAA certificate numbers) based on movie title (after removing "The", making lower case, and substituting all & symbols for "and") along with other data (rating, maybe years).

In [ ]:
main_merge = (
    outer_merge.loc[~((outer_merge['imdb_ids'].isin(imdb_info_d['imdb_ids']))
                    |
                    (outer_merge['mpaa_cert_#'].isin(mpaa_info_d['mpaa_cert_#'])))].reset_index(drop = True)

)

In [ ]:
main_merge.info()

In [ ]:
imdb_dup_certs

In [ ]:
imdbs = pd.concat([imdb_info_d, imdb_dup_certs]).reset_index(drop=True)
mpaas = pd.concat([mpaa_info_d, mpaa_dup_certs]).reset_index(drop=True)

print(len(imdbs))
print(len(mpaas))

In [ ]:
imdbs['match_titles'] = (
    imdbs['imdb_titles']
    .str.lower()
    .str.replace('&', 'and')
    .str.strip('.:!?,')
    .str.strip()
)
imdbs

In [ ]:
imdbs.loc[(imdbs.duplicated(subset=['match_titles', 'release_year'], keep = False))
          &
          (imdbs['rating_reasons'] == 'None')].sort_values('match_titles')#.index

# These appear to either be duplicates or not rated by MPAA
imdbs = imdbs.drop(index = (imdbs.loc[(imdbs.duplicated(subset=['match_titles', 'release_year'], keep = False))
          &
          (imdbs['rating_reasons'] == 'None')].sort_values('match_titles').index)).reset_index(drop=True)

In [ ]:
imdbs.loc[(imdbs.duplicated(subset=['match_titles', 'rating_reasons'], keep = False))
          & 
          (imdbs['rating_reasons'] == 'None')].sort_values('match_titles')

# None of these were rated by the MPAA
imdbs = imdbs.loc[~((imdbs.duplicated(subset=['match_titles', 'rating_reasons'], keep = False))
          & 
          (imdbs['rating_reasons'] == 'None'))].reset_index(drop=True)

In [ ]:
imdbs.loc[(imdbs.duplicated(subset=['match_titles', 'rating_reasons'], keep = False))
         &
          (imdbs['votes'] < 310)]

# These are movies with the same title as movies that were rated, but these were not
imdbs = imdbs.loc[~((imdbs.duplicated(subset=['match_titles', 'rating_reasons'], keep = False))
         &
          (imdbs['votes'] < 310))].reset_index(drop=True)

In [ ]:
mpaas['the_fix'] = mpaas['mpaa_title'].str.endswith(', The')
mpaas['a_fix'] = mpaas['mpaa_title'].str.endswith(', A')
#mpaas['match_titles'] = mpaas['mpaa_title']
mpaas['temp_titles'] = mpaas['mpaa_title'].str.replace(', The', '').str.replace(', A', '')
mpaas.loc[mpaas['the_fix'] == True, 'match_titles'] = 'The ' + mpaas['temp_titles']
mpaas.loc[mpaas['a_fix'] == True, 'match_titles'] = 'A ' + mpaas['temp_titles']
mpaas['match_titles'] = mpaas['match_titles'].fillna(mpaas['mpaa_title'])
mpaas = mpaas.drop(columns = ['the_fix', 'a_fix', 'temp_titles'])
mpaas

In [ ]:
mpaas['match_titles'] = (
    mpaas['match_titles']
    .str.lower()
    .str.replace('&', 'and')
    .str.strip('.:!?,')
    .str.strip()
)
mpaas

In [ ]:
mpaas.loc[(mpaas.duplicated(subset=['match_titles', 'rating_year'], keep = False))
         &
          (mpaas['rating_year'] == '1991')].sort_values('match_titles')

# These movies were all released prior to 1991
mpaas = mpaas.loc[~((mpaas.duplicated(subset=['match_titles', 'rating_year'], keep = False))
         &
          (mpaas['rating_year'] == '1991'))].reset_index(drop=True)

In [ ]:
mpaas.loc[mpaas.duplicated(subset=['match_titles', 'rating_year'], keep = False)].sort_values('match_titles')

In [ ]:
imdbs.loc[imdbs['imdb_titles'] == 'Pirates']

In [ ]:
title_year_rating_reason_match = pd.merge(imdbs, mpaas, left_on = ['match_titles', 'release_year', 'imdb_mpaas', 'rating_reasons'], right_on = ['match_titles', 'rating_year', 'mpaa_rating', 'mpaa_reason'], how = 'inner')
title_year_rating_reason_match['imdb_titles'].value_counts()

In [ ]:
main_merge = pd.concat([main_merge, 
                        title_year_rating_reason_match
                        .drop(columns = ['mpaa_cert_#_y', 'match_titles'])
                        .rename(columns= {'mpaa_cert_#_x':'mpaa_cert_#'})])

In [ ]:
main_merge.columns

In [ ]:
title_year_rating_reason_match.columns

In [ ]:
imdbs.loc[imdbs['imdb_titles'] == 'Alone']

In [ ]:
imdbs_a = imdbs.loc[~imdbs['imdb_ids'].isin(title_year_rating_reason_match['imdb_ids'])].reset_index(drop=True)
len(imdbs_a)

In [ ]:
 mpaas.loc[((mpaas['mpaa_title'].isin(title_year_rating_reason_match['mpaa_title']))
         )]['mpaa_title'].value_counts()

In [ ]:
 mpaas.loc[((mpaas['mpaa_title'].isin(['Resurrection', 'Secret Sins']))
         )]

In [ ]:
title_year_rating_reason_match.loc[((title_year_rating_reason_match['mpaa_title'].isin(['Resurrection', 'Secret Sins']))
         )]

In [ ]:
mpaas_a = mpaas.loc[~((mpaas['mpaa_title'].isin(title_year_rating_reason_match['mpaa_title']))
            &
            (~mpaas['mpaa_cert_#'].isin(['47255', '32891']))
         )].reset_index(drop = True)

In [ ]:
title_year_merge = pd.merge(imdbs_a, mpaas_a, left_on = ['match_titles', 'release_year'], right_on = ['match_titles', 'rating_year'], how = 'inner')
title_year_merge

In [ ]:
main_merge = pd.concat([main_merge, 
                        title_year_merge
                        .drop(columns = ['mpaa_cert_#_y', 'match_titles'])
                        .rename(columns= {'mpaa_cert_#_x':'mpaa_cert_#'})])

In [ ]:
title_year_merge['mpaa_title'].value_counts()

In [ ]:
imdbs_b = imdbs_a.loc[~imdbs_a['imdb_ids'].isin(title_year_merge['imdb_ids'])].reset_index(drop=True)
len(imdbs_b)

In [ ]:
len(mpaas_a.loc[mpaas_a['mpaa_title'].isin(title_year_merge['mpaa_title'])])

In [ ]:
mpaas_b = mpaas_a.loc[~(mpaas_a['mpaa_title'].isin(title_year_merge['mpaa_title']))] 

In [ ]:
pd.merge(imdbs_b, mpaas_b, left_on = ['match_titles', 'rating_reasons'], right_on = ['match_titles', 'mpaa_reason'])['imdb_titles'].value_counts()

In [ ]:
title_reason_merge = pd.merge(imdbs_b, mpaas_b, left_on = ['match_titles', 'rating_reasons'], right_on = ['match_titles', 'mpaa_reason'])

In [ ]:
main_merge = pd.concat([main_merge, 
                        title_reason_merge
                        .drop(columns = ['mpaa_cert_#_y', 'match_titles'])
                        .rename(columns= {'mpaa_cert_#_x':'mpaa_cert_#'})])

In [ ]:
imdbs_c = imdbs_b.loc[~(imdbs_b['imdb_ids'].isin(title_reason_merge['imdb_ids']))].reset_index(drop=True)

In [ ]:
mpaas_b.loc[mpaas_b['mpaa_title'].isin(title_reason_merge['mpaa_title'])]['mpaa_title'].value_counts()

In [ ]:
title_reason_merge.loc[title_reason_merge['mpaa_title'].isin(['Pirates', 'Love Song'])]

In [ ]:
mpaas_b.loc[mpaas_b['mpaa_title'].isin(['Pirates', 'Love Song'])]

In [ ]:
mpaas_c = mpaas_b.loc[~((mpaas_b['mpaa_title'].isin(title_reason_merge['mpaa_title']))
         &
          (~mpaas_b['mpaa_cert_#'].isin(['42996', '52710']))
           )
         ].reset_index(drop=True)

In [ ]:
mpaas_c.loc[mpaas_c.duplicated(subset=['match_titles'], keep=False)]

In [ ]:
# These are all duplicates or not actually rated by MPAA
imdbs_c = imdbs_c.loc[~(imdbs_c.duplicated(subset=['match_titles'], keep=False))]

In [ ]:
imdbs_c.loc[imdbs_c['match_titles'].isin(mpaas_c.loc[mpaas_c.duplicated(subset=['match_titles'], keep=False)]['match_titles'])]

None of the movies that have the same title from the MPAA dataset have matches in the IMDB dataset

In [ ]:
title_rating_match = pd.merge(imdbs_c, mpaas_c, left_on = ['match_titles', 'imdb_mpaas'], right_on = ['match_titles', 'mpaa_rating'])

In [ ]:
title_rating_match = title_rating_match.loc[abs(title_rating_match['release_year'].astype(int) - title_rating_match['rating_year'].astype(int)) < 5]

In [ ]:
title_rating_match

In [ ]:
main_merge = pd.concat([main_merge, 
                        title_rating_match
                        .drop(columns = ['mpaa_cert_#_y', 'match_titles'])
                        .rename(columns= {'mpaa_cert_#_x':'mpaa_cert_#'})]).reset_index(drop=True)

In [ ]:
imdbs_d = imdbs_c.loc[~(imdbs_c['imdb_ids'].isin(title_rating_match['imdb_ids']))].reset_index(drop=True)
len(imdbs_d)

In [ ]:
mpaas_c.loc[mpaas_c['mpaa_title'].isin(title_rating_match['mpaa_title'])]['mpaa_title'].value_counts()

In [ ]:
mpaas_d = mpaas_c.loc[~(mpaas_c['mpaa_title'].isin(title_rating_match['mpaa_title']))]
len(mpaas_d)

In [ ]:
pd.merge(imdbs_d, mpaas_d, left_on = ['match_titles'], right_on = ['match_titles'])

None of the title only "matches" appear to be real matches, not adding them in

In [ ]:
mpaas_d.columns

In [ ]:
final_data = pd.concat([main_merge, mpaas_d])

In [ ]:
final_data['other_info'].str.lower().value_counts()

In [ ]:
final_data = final_data.loc[~(final_data['other_info'].str.lower().str.contains('re-issue'))].reset_index(drop=True)

In [ ]:
# Eliminate duplicate rows that don't have rating reasons from the MPAA site
final_data = final_data.loc[~((final_data.duplicated(subset=['imdb_ids'],keep=False))
               &
               (~final_data['imdb_ids'].isna())
              &
              (final_data['mpaa_reason'] == '')
               & 
              (final_data['mpaa_rating'] != 'G'))]

In [ ]:
# Eliminate multiple rows for the same movie with the exact same rating reason
final_data = final_data.loc[~((final_data.duplicated(subset=['imdb_ids', 'mpaa_reason'], keep='first'))
               &
               (~final_data['imdb_ids'].isna()))].reset_index(drop=True)

In [ ]:
# Decided to drop the "Edited Version" of movies to avoid duplicate financials, etc.
final_data = final_data.loc[~((final_data.duplicated(subset=['imdb_ids'], keep=False))
               &
               (~final_data['imdb_ids'].isna())
               & 
               (final_data['other_info'].str.contains('Edited')))].reset_index(drop=True)

In [ ]:
final_data.loc[((final_data.duplicated(subset=['mpaa_title', 'rating_year', 'mpaa_cert_#'], keep=False))
               #&
               #(final_data['imdb_ids'].isna())
              # & 
               #(final_data['other_info'].str.contains('Edited'))
                )
              ].reset_index(drop=True).sort_values('mpaa_title')

In [ ]:
final_data['the_fix'] = final_data['mpaa_title'].str.endswith(', The')
final_data['a_fix'] = final_data['mpaa_title'].str.endswith(', A')
final_data['temp_titles'] = final_data['mpaa_title'].str.replace(', The', '').str.replace(', A', '')
final_data.loc[final_data['the_fix'] == True, 'clean_titles'] = 'The ' + final_data['temp_titles']
final_data.loc[final_data['a_fix'] == True, 'clean_titles'] = 'A ' + final_data['temp_titles']
final_data['clean_titles'] = final_data['clean_titles'].fillna(final_data['mpaa_title'])
final_data = final_data.drop(columns = ['the_fix', 'a_fix', 'temp_titles'])
final_data.loc[final_data['clean_titles'].str.contains('The')]

In [ ]:
final_data.columns

In [ ]:
final = final_data[['clean_titles', 'rating_year', 'mpaa_rating', 'mpaa_reason',
            'imdb_ratings', 'metascores', 'votes', 'gross', 'release_year', 'release_note', 
             'imdb_runtimes', 'imdb_genres','mpaa_cert_#', 'mpaa_cert_note', 'other_info'
           ]]

In [ ]:
final.to_pickle('../data/final_imdb_mpaa.pkl')